In [ ]:
#@title Imports
import pandas as pd
import numpy as np
from pathlib import Path
import torch
import os
import random
from google.colab import drive
import sys

In [ ]:
#@title here link your goole drive directory that contains the files
MOUNTPOINT = '/content/gdrive'
directoryInsideGoogleDrive = 'HERE YOU DIR NAME'
DATADIR = os.path.join(MOUNTPOINT, 'MyDrive', f{directoryInsideGoogleDrive})
drive.mount(MOUNTPOINT,force_remount=True)

In [ ]:
#@title LSTM class
class LSTM_NN(torch.nn.Module):
  def __init__(self, input_features_num:int, LSTM_hidden_layers_num:int, Dropout:float):
    super(LSTM_NN,self).__init__()
    self._LSTM = torch.nn.LSTM(input_features_num,LSTM_hidden_layers_num,batch_first=True,num_layers=1)
    for names, params in self._LSTM.named_parameters():
      if('weight_in' in names or 'weight_hh' in names):
        torch.nn.init.xavier_uniform_(params.data)
      elif('bias' in names):
        params.data.fill_(0)
    #layer norm od lstm
    self.layer_norm = torch.nn.LayerNorm(LSTM_hidden_layers_num)
    #droput
    self.dropout = torch.nn.Dropout(Dropout)

    # 2 fcc layers inner layers
    self.FCC_1 = torch.nn.Linear(LSTM_hidden_layers_num, LSTM_hidden_layers_num)
    self.layer_norm_fcc1 = torch.nn.LayerNorm(LSTM_hidden_layers_num)

    self.FCC_2 = torch.nn.Linear(LSTM_hidden_layers_num, int(LSTM_hidden_layers_num/2))
    self.layer_norm_fcc2 = torch.nn.LayerNorm(int(LSTM_hidden_layers_num/2))

    #output layer
    self.FCC_3 = torch.nn.Linear(int(LSTM_hidden_layers_num/2),1) #binary classsification

  def forward(self, input_data_tensor:torch.tensor):
      #lstm output
      LSTM_output, _ = self._LSTM(input_data_tensor)
      #lstm output layer norm
      LSTM_output = self.layer_norm(LSTM_output)
      #lstm output dropout
      LSTM_output = self.dropout(LSTM_output)

      #fcc1
      output = self.FCC_1(LSTM_output[:,-1,:])
      output = self.layer_norm_fcc1(output)
      output = torch.tanh(output)
      output = self.dropout(output)

      #fcc2
      output = self.FCC_2(output)
      output = self.layer_norm_fcc2(output)
      output = torch.tanh(output)
      output = self.dropout(output)

      #fcc3
      output = self.FCC_3(output)
      output = torch.sigmoid(output).view(-1,1,1)

      return output

In [ ]:
#@title shuffle data, needs same seed for train, validation and test, default seed is enough
def shufflePathsAndSplitForTrainValidationtest(dirNameWithFiles:str,seed:int = 234):
    #returns a list of random paths to the files for training, validation and test
    #vraca listu random pathova za train, validate i test
    random.seed(seed)
    listOfPaths = os.listdir(os.path.join(DATADIR,f"{dirNameWithFiles}"))
    listOfPaths = [os.path.join(DATADIR,f"{dirNameWithFiles}/{fileName}") for fileName in listOfPaths]
    random.shuffle(listOfPaths)
    #split  80 , 10 ,10
    totalNumberOfFiles = len(listOfPaths)
    trainListOfPaths = listOfPaths[:int(totalNumberOfFiles*0.8)]
    validateListOfPaths = listOfPaths[int(totalNumberOfFiles*0.8):int(totalNumberOfFiles*0.9)]
    testListOfPaths = listOfPaths[int(totalNumberOfFiles*0.9):]
    #return
    return trainListOfPaths, validateListOfPaths, testListOfPaths

In [ ]:
#@title mean i std used for data normalization
def GetMeanAndStdOdTrainData(TrainListOfPaths:list[str,str], NumberOfFeatures:int = 3):
  #returns mean and std of train data
  #Welford's method
  n = 0
  mean = np.zeros(NumberOfFeatures)
  M2 = np.zeros(NumberOfFeatures)
  for path in TrainListOfPaths:
      trainData = pd.read_csv(path,header=None)
      trainData = trainData.to_numpy()
      #all measurements into one row
      trainData = (trainData[:,:NumberOfFeatures])
      totalNumberOfMeasurements = trainData.shape[0] #sequence number
      for i in range(totalNumberOfMeasurements):
        n += 1
        delta = trainData[i,:] - mean
        mean += delta / n
        delta2 = trainData[i,:] - mean
        M2 += (delta * delta2)

  variance = M2 / (n - 1)
  std = np.sqrt(variance)

  return mean, std

In [ ]:
#@title Custom DataLoader
def CustomDataLoader(CurrentBatchFilePaths:list[str,str],cpuTest = False,FeaturesNumber:int=3):
    #loads a list that contains a current batch paths to the csv files and then creates tensors
    #for them ordered to keep the label and train pairs ordering
    listOfInputsMeasurements = []
    listOfLabelsMeasurements = []
    for csvMeasurementPath in CurrentBatchFilePaths:
        TempDataframe = pd.read_csv(csvMeasurementPath,header=None)
        #fill the list with third dimencion for later stacking
        TempNumpyArray = TempDataframe.to_numpy()
        TempNumpyArray = TempNumpyArray[None,:,:]
        listOfInputsMeasurements.append(TempNumpyArray[:,:,:FeaturesNumber]) #all rows, columns 1,2,3
        listOfLabelsMeasurements.append(TempNumpyArray[:,0,3].reshape(1,1,1)) #first row and column 4
    #stack
    InputData = np.concatenate(listOfInputsMeasurements,axis=0)
    LabelsData = np.concatenate(listOfLabelsMeasurements,axis=0)
    #witch to Torch
    InputData = torch.tensor(InputData, dtype = torch.float32)
    LabelsData = torch.tensor(LabelsData, dtype = torch.float32)
    #device select, force cuda
    if(torch.cuda.is_available() == True):
        InputData.to('cuda')
        LabelsData.to('cuda')
    elif(torch.cuda.is_available() == False and cpuTest==False):
        raise Exception("Cuda not avaliable!!")
    #return input first and then labels
    return InputData, LabelsData

In [ ]:
#@title heleper function for later saving the model and loss results
def SaveTrainValidationLoss(train_losses, validation_losses):
    'train'
    train_losses_dataframe = pd.DataFrame(train_losses,columns=['losses'])
    CsvPathForTrainLosses=os.path.join(DATADIR,f'Model/Train_Losses.csv')
    #check if exist alerady to append to or make new
    if not pd.io.common.file_exists(CsvPathForTrainLosses):
        train_losses_dataframe.to_csv(CsvPathForTrainLosses, index=False)
    else:
        train_losses_dataframe.to_csv(CsvPathForTrainLosses,mode='a',header=False,index=False)

    'validation'
    validation_losses_dataframe = pd.DataFrame(validation_losses,columns=['validation'])
    CsvPathForValidationLosses=os.path.join(DATADIR,'Model/Validation_Losses.csv')
    if not pd.io.common.file_exists(CsvPathForValidationLosses):
        validation_losses_dataframe.to_csv(CsvPathForValidationLosses, index=False)
    else:
        validation_losses_dataframe.to_csv(CsvPathForValidationLosses,mode='a',header=False,index=False)

In [ ]:
#@title Main Training function
def TrainNetwork(FeaturesNumber:int, LSTM_hidden_num:int, Dropout_Value:int,
                 epoch_num:int, batch_size:int,
                 LossFunction = torch.nn.BCELoss, Optimizer = torch.optim.Adam, l2=1e-6,
                 learning_rate:float = 0.1,
                 exponential_learning_rate_scheduler_gamma = 0.99,
                 max_grad_clip_value = 10.0,
                 validation_patience : int = 100,
                 precalculated_mean_std:bool = False,
                 precalculated_mean:torch.tensor = None ,
                 precalculated_std:torch.tensor = None,
                 continueToTrain : bool = False,
                 dirNameWithFiles:str=None):
    #using classification LOSS Binary Cross Entropy
    #can use precalculated mena and std to speed up parts
    if (torch.cuda.is_available() == True):
        device= "cuda"
    else:
        raise Exception('Cuda not properly setup!!')
    #check for directory for model and metadata and if notexistanta create it
    if(os.path.isdir("content/Model") == False):
        os.mkdir(os.path.join(DATADIR,'Model'))

    #load existing model or make new
    if(continueToTrain == True):
        print('Continuing to train')
        modelPath = os.path.join(DATADIR,'Model/ModelWithMetadata.pth')
        Checkpoint = torch.load(modelPath)
        model = LSTM_NN(input_features_num = Checkpoint['input_features_num'],
                        LSTM_hidden_layers_num= Checkpoint['LSTM_hidden_layers_num'],
                        Dropout = Checkpoint['dropout'])
        model.load_state_dict(Checkpoint    ['model_state_dict'])
        model = model.to(device)
    else:
        'CREATING LSTM Model'
        model = LSTM_NN(input_features_num = FeaturesNumber,
                        LSTM_hidden_layers_num = LSTM_hidden_num,
                        Dropout = Dropout_Value)
        model = model.to(device)
    #loss
    selected_loss = LossFunction().to(device)
    #randomize and split data
    train_data_CSV_path, validate_data_CSV_path, test_data_CSV_path =\
                             shufflePathsAndSplitForTrainValidationtest(dirNameWithFiles)
    #mean and std for normalization
    if(precalculated_mean_std == False):
      print('creating --> 80,10,10 <-- split and calcuating Mean and Std of train data')
      mean,std = GetMeanAndStdOdTrainData(train_data_CSV_path, FeaturesNumber)
      mean,std = torch.tensor(mean, dtype = torch.float32), torch.tensor(std, dtype = torch.float32)
      mean,std = mean.to(device), std.to(device)
      print('mean')
      print(mean)
      print('std')
      print(std)
      print('done')
    else:
      mean,std = precalculated_mean.to(device), precalculated_std.to(device)
    print()
    print('Training:')
    TrainingLosses = []
    ValidationLosses = []
    previousValidationLoss = float('inf')
    improvementsTracker = 0
    selected_optimizer = Optimizer(model.parameters(),lr=learning_rate, weight_decay=l2)
    #or use decay that is commented
    #selected_learning_rate_scheduler = torch.optim.lr_scheduler.ExponentialLR(selected_optimizer,
    #                                              gamma= exponential_learning_rate_scheduler_gamma)
    for epoch in range(epoch_num):
        print(f'==== EPOCH: {epoch} ====')
        ############ train mode #############
        'switching into training mode'
        model.train()
        totalTrainLoss = 0
        num_batches = 0
        random.shuffle(train_data_CSV_path)
        random.shuffle(validate_data_CSV_path)
        #iterating over a list of paths and evokuing custom dataloader
        for batch_index in range(0,len(train_data_CSV_path),batch_size):
            'loading training batch and creating formated tensor data and labels'
            torch.cuda.empty_cache()
            data,labels= \
                CustomDataLoader(train_data_CSV_path[batch_index:batch_index+velicina_batcha],FeaturesNumber=FeaturesNumber)
            data,labels = data.to(device), labels.to(device)
            'normalize data'
            data = (data - mean)/std
            'forward propagation'
            model.zero_grad()
            output = model(data)
            loss = selected_loss(output,labels)
            totalTrainLoss += loss.item()
            'backward propagation'
            loss.backward()
            'clip gradient'
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_clip_value)
            'weight update'
            selected_optimizer.step()
            print(f'\r---> LOSS: {loss.item()} ',end='')
            #increase number of batches tracker
            num_batcheva += 1
        #average value through the epoch
        AverageTrainingLoss = totalTrainLoss/broj_batcheva
        TrainingLosses.append(AverageTrainingLoss)
        #if decide to use scheduler was selected uncomment next line
        #selected_learning_rate_scheduler.step()


        ########## evaluation mode ############
        'switching into evaluation mode'
        model.eval()
        TotalValidationLoss = 0
        num_batches = 0
        with torch.no_grad():
            for batch_index in range(0,len(validate_data_CSV_path),batch_size):
                'loading validtion batch and creating formated tensor data and labels'
                torch.cuda.empty_cache()
                data,labels= \
                    DataLoaderKojiVracaTorchTensorInputaILabela(validate_data_CSV_path[batch_index:batch_index+batch_size])
                data,labels = data.to(device), labels.to(device)
                'normalize'
                data = (data - mean)/std
                output = model(data)
                loss = selected_loss(output,labels)
                TotalValidationLoss += loss.item()
                ValidationLosses.append(TotalValidationLoss)
                num_batches += 1
            #epoch average
            print()
            AverageValidationLoss = TotalValidationLoss/num_batches
            ValidationLosses.append(AverageValidationLoss)
            print(f'==== Results of epoch {epoch} :Average Training LOSS --> {AverageTrainingLoss:.12f},Average Validation LOSS --> {AverageValidationLoss:.12f} ====')
            'early stop check'
            if(np.round(AverageValidationLoss,4) >= np.round(previousValidationLoss,4)):
                improvementsTracker += 1
                if(improvementsTracker >= validation_patience):
                    print('Early Stop')
                    break
            else:
                previousValidationLoss = AverageValidationLoss

    'Metadata saving'
    print()
    print('Saving Model, Model Metadata, train and validation loss')
    SaveTrainValidationLoss(TrainingLosses, ValidationLosses)
    torch.save({
            'input_features_num': FeaturesNumber,
            'LSTM_hidden_layers_num': LSTM_hidden_num,
            'dropout': Dropout_Value,
            'model_state_dict': model.state_dict()
            },os.path.join(DATADIR,'Model/ModelWithMetadata.pth'))
    print('done')

In [ ]:
#@title Test the network
def testNN(FeaturesNumber,
           batch_size:int,
           separate_test_dir:bool,
           precalculated_mean_std:bool = False,
           precalculated_mean:torch.tensor = None ,
           precalculated_std:torch.tensor = None):
    if (torch.cuda.is_available() == True):
        device = "cuda"
    else:
        raise Exception('Cuda nije dobro postavljena')
    PathDoModela = os.path.join(DATADIR,'Model/ModelWithMetadata.pth')
    Checkpoint = torch.load(PathDoModela)
    model = LSTM_NN(input_features_num = Checkpoint['input_features_num'],
                    LSTM_hidden_layers_num= Checkpoint['LSTM_hidden_layers_num'],
                    Dropout = Checkpoint['dropout'])
    model.load_state_dict(Checkpoint['model_state_dict'])
    model = model.to(device)
    if(zasebni_test_folder == True):
      test_data_CSV_path = os.listdir(os.path.join(DATADIR,"SeparateDir"))
      test_data_CSV_path = [os.path.join(DATADIR,f"SeparateDir/{fileName}") for fileName in test_data_CSV_path]
    else:
      train_data_CSV_path, _ , test_data_CSV_path = shufflePathsAndSplitForTrainValidationtest(dirNameWithFiles)
    #determine mean std on train data
    if(precalculated_mean_std == False):
      print('creating --> 80,10,10 <-- split and calcuating Mean and Std of train data')
      mean,std = GetMeanAndStdOdTrainData(train_data_CSV_path, FeaturesNumber)
      mean,std = torch.tensor(mean, dtype = torch.float32), torch.tensor(std, dtype = torch.float32)
      mean,std = mean.to(device), std.to(device)
      print('mean')
      print(mean)
      print('std')
      print(std)
      print('done')
    else:
      mean,std = precalculated_mean.to(device), precalculated_std.to(device)
    model.eval()
    print()
    print("TESTING: ")
    Labels_List_For_Whole_Column = []
    Output_List_For_Whole_Column = []
    with torch.no_grad():
        for batch_index in range(0,len(test_data_CSV_path),batch_size):
            'loading test batch and creating formated tensor data and labels'
            data,labels= \
                CustomDataLoader(test_data_CSV_path[batch_index:batch_index+batch_size],FeaturesNumber)
            data,labels = data.to(device), labels.to(device)
            'normalize'
            data = (data - mean)/std
            output = model(data)
            #reshe for output
            labels_reshaped = labels.to('cpu')
            labels_reshaped = labels_reshaped.numpy()
            labels_reshaped = labels_reshaped.reshape(-1,1)
            output_reshaped = output.to('cpu')
            output_reshaped = output_reshaped.numpy()
            output_reshaped = output_reshaped.reshape(-1,1)
            Labels_List_For_Whole_Column.append(labels_reshaped)
            Output_List_For_Whole_Column.append(output_reshaped)
    #save .csv concatenated
    LabelsNumpy = np.concatenate(Labels_List_For_Whole_Column)
    OutputNumpy = np.concatenate(Output_List_For_Whole_Column)
    Output = np.hstack((LabelsNumpy,OutputNumpy))
    Output = pd.DataFrame(Output, columns = ['LABELS', 'PREDICTIONS'])
    if(zasebni_test_folder == True):
      PathZaSejv = os.path.join(DATADIR,'SeparateDir/Results.csv')
    else:
      PathZaSejv = os.path.join(DATADIR,'Model/Results.csv')
    Output.to_csv(PathZaSejv,columns=['LABELS','PREDICTIONS'])
    print('Saved --> Results.csv with labels and predictions inside Model directory !!!')

In [ ]:
#@title MAIN

#in case of startaing anew
if(os.path.isdir(os.path.join(DATADIR,'Model')) == True):
        os.rmdir(os.path.join(DATADIR,'Model'))
torch.cuda.empty_cache()
#here i just stored precalculted mean std
mean = torch.tensor([ 0.0011, -0.0023,  0.0020], dtype = torch.float32)
std = torch.tensor([0.0242, 0.0273, 0.0212], dtype = torch.float32)
#train
TrainNetwork(FeaturesNumber = 3, LSTM_hidden_num = 300 ,Dropout_Value=0.1, epoch_num = 100,validation_patience=100,
             batch_size = 5, learning_rate = .00001,
             precalculated_mean_std = True, precalculated_mean = mean, precalculated_std = std)

#test
TestiranjeMreze(FeaturesNumber = 3, batch_size = 50,separate_test_dir = False,
                precalculated_mean_std = False, precalculated_mean = mean, precalculated_std = std)